In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import time
import copy
import string
import json
import pickle as pk

from collections import Counter

from math import sin, asin, cos, radians, fabs, sqrt

from sklearn.cluster import KMeans,MiniBatchKMeans,AgglomerativeClustering,SpectralClustering
from sklearn.datasets import make_blobs

############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()


In [2]:
####################   proba()  把 staying list  转换为概率特征    ############
#  输入: staying list   一个POI上的停留时间list ;  把停留时间化为了9个时段0::8
#  输出: st_f      停留时间概率分布，  
def proba(st):
    t = st.copy()
    for i in range(0, len(st)):
    
        if 10 <= st[i] < 30:
            t[i] = 0
        elif 30 <= st[i] < 60:
            t[i] = 1
        elif 60 <= st[i] < 90:
            t[i] = 2
        elif 90 <= st[i] < 120:
            t[i] = 3
        elif 120 <= st[i] < 150:
            t[i] = 4
        elif 150 <= st[i] < 180:
            t[i] = 5
        elif 180 <= st[i] < 210:
            t[i] = 6
        elif 210 <= st[i] < 240:
            t[i] = 7
        else:
            t[i] = 8
    
    b = len(t)
    a = Counter(t)
    #print(a)
    stf = [0]*9        # staying list 转化为 fecture vector
    #print(st_f)

    for key, value in a.items():
        stf[key] = round(value/b, 3)
        
    return stf   

In [ ]:
########################  读取 st_poi_tree ########################
##   {"enter"}{"bar"}[{},{},{}]
filename = ".\\os_file\\st_poi_tree.pkl"        
st_poi_tree = read_q(filename)
st_poi_tree["enter"]["bar"]


In [ ]:
#######################  统计总长度      #########################

a = 0
for item1 in st_poi_tree:
    print("第一大类 ", item1)
    b = 0
    
    for item2 in st_poi_tree[item1]:
        a += len(st_poi_tree[item1][item2])

print("共有PoI： ",a,"个" )

In [3]:
######################   mb_kmeans  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic  [[{},{},{}...],[{},{},{}...],[],[]]
def mb_kmeans(list_dic, n):
    

    print("调用 mb_kmeans，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = MiniBatchKMeans(n_clusters = n ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []     # 为每个聚类建立一个列表
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d

In [21]:
######################   a_cluster  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic
def a_cluster(list_dic, n):
    

    print("调用 a_cluster，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = AgglomerativeClustering(n_clusters = n ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d

In [3]:
######################   s_cluster  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic
def s_cluster(list_dic, n):
    

    print("调用 s_cluster，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = SpectralClustering(n_clusters = n,affinity = "nearest_neighbors" ).fit_predict(X)
    #y_pred = spectral_clustering( ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d

In [3]:
######################   k_means  聚类 分类   #####################
#输入 list_dic     [{},{},{}]  POI_list
#输出 l_l_d   list_list_dic
def k_means(list_dic, n):
    

    print("调用 k-means，聚类前list的长度为： ", len(list_dic))
    l_l_d = []
    a = []
    for i in range(0, len(list_dic)):
        b = list_dic[i]["staying"]
        c = proba(b)
        a.append(c)
        

    X = np.array(a)
    print("参与聚类的list 的长度", len(X))
    y_pred = KMeans(n_clusters = n ).fit_predict(X)
    d = 0    
    for j in range(n):
            
        a1 = []
        for k in range(0, len(X)):
                 
            if y_pred[k] == j:
                    
                a1.append(list_dic[k])
        print("聚类后，第一类list的长度为： ",len(a1))
        d += len(a1)
        l_l_d.append(a1)
        
    print("聚类后，两层列表总长度为： ", d)
    
    return l_l_d

[{'id': '大堂吧',
  'location': {'lat': 39.996445, 'lng': 116.312735},
  'staying': [77.04999993788078],
  'category': 'bar'},
 {'id': '京都信苑饭店-大堂酒吧',
  'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
  'staying': [12.249999940395355],
  'category': 'bar'},
 {'id': '半山酒吧',
  'location': {'lat': 39.973151, 'lng': 116.48791299999999},
  'staying': [35.41666665812954, 151.58333333325572],
  'category': 'bar'},
 {'id': 'juice bar',
  'location': {'lat': 39.918248, 'lng': 116.31093600000001},
  'staying': [42.083333290647715, 151.28333336790092],
  'category': 'bar'},
 {'id': '酒吧',
  'location': {'lat': 39.975998, 'lng': 116.32942800000001},
  'staying': [14.19999997713603,
   20.000000012805685,
   20.016666578594595,
   14.400000002933666,
   10.516666610492393,
   12.366666707675904,
   46.233333349227905,
   20.39999990724027,
   14.499999932013452,
   14.466666674707085,
   109.61666668416001,
   90.96666666213423,
   141.46666675340384,
   31.41666661365889,
   48.0166

In [6]:
#####################  第一次 cluster   ###############################
# n    聚类个数
filename = ".\\os_file\\st_poi_clusters\\mb_kmeans\\st_poi_cluster_1st.pkl"
n = 4
list_dic = []
st_poi_kmeans1 = copy.deepcopy(st_poi_tree)
a = 0
for item1 in st_poi_tree:
    print("第一大类 ", item1)
    b = 0
    
    for item2 in st_poi_tree[item1]:
       
        list_dic = st_poi_tree[item1][item2]
        print("    当前小类： ", item2, "POI长度为： ",len(list_dic))
        b += len(list_dic)
        
        if len(list_dic) < 10:
            continue
        
        k_m = mb_kmeans(list_dic, n)    
       
        st_poi_kmeans1[item1][item2] = k_m
    print("第一大类： ",item1,"共有POI：",b, "个")
    
    a += b    
print("共处理POI： ",a," 个")
write_q(st_poi_kmeans1, filename)

第一大类  enter
    当前小类：  bar POI长度为：  27
调用 mb_kmeans，聚类前list的长度为：  27
参与聚类的list 的长度 27
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  18
聚类后，第一类list的长度为：  6
聚类后，第一类list的长度为：  2
聚类后，两层列表总长度为：  27
    当前小类：  bath POI长度为：  124
调用 mb_kmeans，聚类前list的长度为：  124
参与聚类的list 的长度 124
聚类后，第一类list的长度为：  29
聚类后，第一类list的长度为：  75
聚类后，第一类list的长度为：  9
聚类后，第一类list的长度为：  11
聚类后，两层列表总长度为：  124
    当前小类：  bbulid POI长度为：  356
调用 mb_kmeans，聚类前list的长度为：  356
参与聚类的list 的长度 356
聚类后，第一类list的长度为：  95
聚类后，第一类list的长度为：  33
聚类后，第一类list的长度为：  211
聚类后，第一类list的长度为：  17
聚类后，两层列表总长度为：  356
    当前小类：  cinema POI长度为：  2
    当前小类：  farm POI长度为：  7
    当前小类：  ktv POI长度为：  38
调用 mb_kmeans，聚类前list的长度为：  38
参与聚类的list 的长度 38
聚类后，第一类list的长度为：  28
聚类后，第一类list的长度为：  6
聚类后，第一类list的长度为：  3
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  38
    当前小类：  scenic POI长度为：  515
调用 mb_kmeans，聚类前list的长度为：  515
参与聚类的list 的长度 515
聚类后，第一类list的长度为：  280
聚类后，第一类list的长度为：  49
聚类后，第一类list的长度为：  165
聚类后，第一类list的长度为：  21
聚类后，两层列表总长度为：  515
    当前小类：  tea POI长度为：  8
第一大类：  ente

In [7]:
#############################  2次  cluster  ######################
#  输入 st_poi_kmeans1
#  输出 st_poi_kmeans2
filename = ".\\os_file\\st_poi_clusters\\mb_kmeans\\st_poi_cluster_2nd.pkl"
n = 4
#list_dict = []
st_poi_kmeans2 = copy.deepcopy(st_poi_kmeans1)
a = 0
for item1 in st_poi_kmeans1:
    print("第一大类： ",item1)
    
    a1= 0
    for item2 in st_poi_kmeans1[item1]:
        b = 0
        st = st_poi_kmeans1[item1][item2]
        print("    当前小类： ",item2)
        if 0 <= len(st)< 4 or 4 < len(st) < 10:  #未动过的点
            b += len(st)
            a1 += b
            continue
        
        if type(st[0]).__name__ == 'dict':  # len =4 的未动过的点  根据下一级对象类型
            b += len(st)
            a1 += b
            continue
        
        #type(st[0]).__name__ == 'list':
        
        for i in range(len(st)):    
            list_dict = st[i]
            b += len(list_dict)
            
            if len(list_dict) < 10:
                continue
            
            k_m = mb_kmeans(list_dict, n)
            
            st_poi_kmeans2[item1][item2][i] = k_m
        
        a1 += b
        print(item2,"共处理POI： ",b,"个")
    a += a1
    print(item1,"共处理POI： ",a1,"个")
print("第二次K-means 共处理POI：",a,"个")
            
write_q(st_poi_kmeans2,filename)

第一大类：  enter
    当前小类：  bar
调用 mb_kmeans，聚类前list的长度为：  18
参与聚类的list 的长度 18
聚类后，第一类list的长度为：  16
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  18
bar 共处理POI：  27 个
    当前小类：  bath
调用 mb_kmeans，聚类前list的长度为：  29
参与聚类的list 的长度 29
聚类后，第一类list的长度为：  2
聚类后，第一类list的长度为：  8
聚类后，第一类list的长度为：  17
聚类后，第一类list的长度为：  2
聚类后，两层列表总长度为：  29
调用 mb_kmeans，聚类前list的长度为：  75
参与聚类的list 的长度 75
聚类后，第一类list的长度为：  61
聚类后，第一类list的长度为：  7
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  6
聚类后，两层列表总长度为：  75
调用 mb_kmeans，聚类前list的长度为：  11
参与聚类的list 的长度 11
聚类后，第一类list的长度为：  10
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  11
bath 共处理POI：  124 个
    当前小类：  bbulid
调用 mb_kmeans，聚类前list的长度为：  95
参与聚类的list 的长度 95
聚类后，第一类list的长度为：  78
聚类后，第一类list的长度为：  8
聚类后，第一类list的长度为：  5
聚类后，第一类list的长度为：  4
聚类后，两层列表总长度为：  95
调用 mb_kmeans，聚类前list的长度为：  33
参与聚类的list 的长度 33
聚类后，第一类list的长度为：  32
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  33
调用 mb_kmeans，聚类前list的长度为

In [4]:
filename = ".\\os_file\\st_poi_clusters\\k_means\\st_poi_cluster_2nd.pkl"
st_poi_kmeans2 = read_q(filename)
st_poi_kmeans2["enter"]["bar"]

[[[{'id': '京都信苑饭店-大堂酒吧',
    'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
    'staying': [12.249999940395355],
    'category': 'bar'},
   {'id': '顽啤猴精酿(西山摩尔汇店)',
    'location': {'lat': 39.934197, 'lng': 116.103996},
    'staying': [19.033333350671455],
    'category': 'bar'},
   {'id': '北京中关村皇冠假日酒店-大堂酒廊',
    'location': {'lat': 39.981415999999996, 'lng': 116.329609},
    'staying': [23.7500000721775, 10.083333301590756],
    'category': 'bar'},
   {'id': '九霄云外酒吧(香格里拉饭店)',
    'location': {'lat': 39.950293, 'lng': 116.31490600000001},
    'staying': [14.500000078696758],
    'category': 'bar'},
   {'id': 'seven bar',
    'location': {'lat': 39.948612, 'lng': 116.40956399999999},
    'staying': [11.916666717734188],
    'category': 'bar'},
   {'id': '星巴克(腾达大厦店)',
    'location': {'lat': 39.943732000000004, 'lng': 116.333277},
    'staying': [14.949999932432547],
    'category': 'bar'},
   {'id': '万欧兰葡萄酒俱乐部',
    'location': {'lat': 39.905714, 'lng': 116.475268},


In [5]:
#############################  3次  cluster   ######################
#  输入 st_poi_kmeans1
#  输出 st_poi_kmeans2

filename = ".\\os_file\\st_poi_clusters\\k_means\\st_poi_cluster_3nd.pkl"
n = 4
#list_dict = []
st_poi_kmeans3 = copy.deepcopy(st_poi_kmeans2)
a = 0
for item1 in st_poi_kmeans2:
    print("当前大类： ",item1)
    
    a1= 0
    for item2 in st_poi_kmeans2[item1]:    #  第一层： 判断结点类型，是原始结点，还是生成结点
        b = 0
        st_1 = st_poi_kmeans2[item1][item2]       
        print("    当前小类： ",item2)
        if 0 <= len(st_1)< 4 or 4 < len(st_1) < 10:  #未动过的点
            b += len(st_1)
            a1 += b
            continue
        
        if type(st_1[0]).__name__ == 'dict':  # len =4 的未动过的点  根据下一级对象类型
            b += len(st_1)
            a1 += b
            continue
        
        #type(st[0]).__name__ == 'list':
        
        for i in range(len(st_1)):               #  第二层：    对第一次聚类生成的结点，进行第二次聚类操作
    
            #########################################  
            st_2 = st_1[i]
            #print("    当前小类： ",item2)
            if 0 <= len(st_2)< 4 or 4 < len(st_2) < 10:  #  第二次聚类 不动的点
                b += len(st_2)
                #a1 += b
                continue
        
            if type(st_2[0]).__name__ == 'dict':  # len =4 的未动过的点  根据下一级对象类型
                b += len(st_2)
                #a1 += b
                continue
        
            #type(st[0]).__name__ == 'list':
        
            for j in range(len(st_2)):       # 第三层， 对第二次聚类生成的结点进行第三次聚类操作
                st_3 = st_2[j]
        
                b += len(st_3)
                #a1 += b
            
                if len(st_3) < 10:
                    continue
            
                k_m = k_means(st_3, n)
            
                st_poi_kmeans3[item1][item2][i][j] = k_m
            
            ############################################
        
        a1 += b
        print(item2,"共处理POI： ",b,"个")
    a += a1
    print(item1,"共处理POI： ",a1,"个")
print("第三次K-means 共处理POI：",a,"个")
            
write_q(st_poi_kmeans3,filename)

当前大类：  enter
    当前小类：  bar
调用 k-means，聚类前list的长度为：  16
参与聚类的list 的长度 16
聚类后，第一类list的长度为：  16
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  16
bar 共处理POI：  27 个
    当前小类：  bath
调用 k-means，聚类前list的长度为：  10
参与聚类的list 的长度 10
聚类后，第一类list的长度为：  10
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  10
调用 k-means，聚类前list的长度为：  61
参与聚类的list 的长度 61
聚类后，第一类list的长度为：  61
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  61
调用 k-means，聚类前list的长度为：  20
参与聚类的list 的长度 20
聚类后，第一类list的长度为：  4
聚类后，第一类list的长度为：  11
聚类后，第一类list的长度为：  3
聚类后，第一类list的长度为：  2
聚类后，两层列表总长度为：  20
bath 共处理POI：  124 个
    当前小类：  bbulid
调用 k-means，聚类前list的长度为：  177
参与聚类的list 的长度 177
聚类后，第一类list的长度为：  173
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  2
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  177
调用 k-means，聚类前list的长度为：  18
参与聚类的list 的长度 18
聚类后，第一类list的长度为：  8
聚类后，第一类list的长度为：  3
聚类后，第一类list的长度为：  5
聚类后，第一类list的长度为：  2
聚类后，两层列表总长度为：  18
调用 k-means，聚类前list的长度为：  11
参与聚类的

聚类后，第一类list的长度为：  44
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  44
调用 k-means，聚类前list的长度为：  19
参与聚类的list 的长度 19
聚类后，第一类list的长度为：  19
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  19
train 共处理POI：  568 个
    当前小类：  univer
调用 k-means，聚类前list的长度为：  10
参与聚类的list 的长度 10
聚类后，第一类list的长度为：  3
聚类后，第一类list的长度为：  7
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  10
调用 k-means，聚类前list的长度为：  10
参与聚类的list 的长度 10
聚类后，第一类list的长度为：  4
聚类后，第一类list的长度为：  2
聚类后，第一类list的长度为：  3
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  10
调用 k-means，聚类前list的长度为：  52
参与聚类的list 的长度 52
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  51
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  52
univer 共处理POI：  112 个
school 共处理POI：  808 个
当前大类：  shop
    当前小类：  bookstore
调用 k-means，聚类前list的长度为：  20
参与聚类的list 的长度 20
聚类后，第一类list的长度为：  20
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  20
bookstore 共处理POI：  38 个
    当前小类：  fmarket
调用 k-means，聚类前list的长度为： 

In [6]:
#############################  4次  cluster   ######################
#  输入 st_poi_kmeans1
#  输出 st_poi_kmeans2

filename = ".\\os_file\\st_poi_clusters\\k_means\\st_poi_cluster_4nd.pkl"
n = 4
#list_dict = []
st_poi_kmeans4 = copy.deepcopy(st_poi_kmeans3)
a = 0
for item1 in st_poi_kmeans3:
    print("当前大类： ",item1)
    
    a1= 0
    for item2 in st_poi_kmeans3[item1]:    #  第一层： 判断结点类型，是原始结点，还是生成结点
        b = 0
        st_1 = st_poi_kmeans3[item1][item2]       
        print("    当前小类： ",item2)
        if 0 <= len(st_1)< 4 or 4 < len(st_1) < 10:  #未动过的点
            b += len(st_1)
            a1 += b
            continue
        
        if type(st_1[0]).__name__ == 'dict':  # len =4 的未动过的点  根据下一级对象类型
            b += len(st_1)
            a1 += b
            continue
        
        #type(st[0]).__name__ == 'list':
        
        for i in range(len(st_1)):               #  第二层：    对第一次聚类生成的结点，进行第二次聚类操作
    
            st_2 = st_1[i]
            #print("    当前小类： ",item2)
            if 0 <= len(st_2)< 4 or 4 < len(st_2) < 10:  #  第二次聚类 不动的点
                b += len(st_2)
                #a1 += b
                continue
        
            if type(st_2[0]).__name__ == 'dict':  # len =4 的未动过的点  根据下一级对象类型
                b += len(st_2)
                #a1 += b
                continue
        
            #type(st[0]).__name__ == 'list':
        
            for j in range(len(st_2)):       # 第三层， 对第二次聚类生成的结点进行第三次聚类操作
                
                ###################################################3
                st_3 = st_2[j]
                #print("    当前小类： ",item2)
                if 0 <= len(st_3)< 4 or 4 < len(st_3) < 10:  #  第三次聚类 不动的点
                    b += len(st_3)
                    #a1 += b
                    continue
        
                if type(st_3[0]).__name__ == 'dict':  # len =4 的未动过的点  根据下一级对象类型
                    b += len(st_3)
                    #a1 += b
                    continue
        
            #type(st[0]).__name__ == 'list':
        
                for k in range(len(st_3)):       # 第四层， 对第三次聚类生成的结点进行第四次聚类操作
                    st_4 = st_3[k]
        
                    b += len(st_4)
                    #a1 += b
            
                    if len(st_4) < 10:
                        continue
            
                    k_m = k_means(st_4, n)
            
                    st_poi_kmeans4[item1][item2][i][j][k] = k_m

                ############################################################

        
        a1 += b
        print(item2,"共处理POI： ",b,"个")
    a += a1
    print(item1,"共处理POI： ",a1,"个")
print("第四次K-means 共处理POI：",a,"个")
            
write_q(st_poi_kmeans4,filename)

当前大类：  enter
    当前小类：  bar
调用 k-means，聚类前list的长度为：  16
参与聚类的list 的长度 16
聚类后，第一类list的长度为：  16
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  16
bar 共处理POI：  27 个
    当前小类：  bath
调用 k-means，聚类前list的长度为：  10
参与聚类的list 的长度 10
聚类后，第一类list的长度为：  10
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  10
调用 k-means，聚类前list的长度为：  61
参与聚类的list 的长度 61
聚类后，第一类list的长度为：  61
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  61
调用 k-means，聚类前list的长度为：  11
参与聚类的list 的长度 11
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  5
聚类后，第一类list的长度为：  4
聚类后，第一类list的长度为：  1
聚类后，两层列表总长度为：  11
bath 共处理POI：  124 个
    当前小类：  bbulid
调用 k-means，聚类前list的长度为：  173
参与聚类的list 的长度 173
聚类后，第一类list的长度为：  173
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  173
调用 k-means，聚类前list的长度为：  32
参与聚类的list 的长度 32
聚类后，第一类list的长度为：  32
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  32
调用 k-means，聚类前list的长度为：  32
参与聚类的

聚类后，第一类list的长度为：  20
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  20
bookstore 共处理POI：  38 个
    当前小类：  fmarket
调用 k-means，聚类前list的长度为：  13
参与聚类的list 的长度 13
聚类后，第一类list的长度为：  13
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  13
fmarket 共处理POI：  20 个
    当前小类：  mall
    当前小类：  smarket
调用 k-means，聚类前list的长度为：  28
参与聚类的list 的长度 28
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  1
聚类后，第一类list的长度为：  26
聚类后，两层列表总长度为：  28
调用 k-means，聚类前list的长度为：  48
参与聚类的list 的长度 48
聚类后，第一类list的长度为：  48
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  48
调用 k-means，聚类前list的长度为：  17
参与聚类的list 的长度 17
聚类后，第一类list的长度为：  3
聚类后，第一类list的长度为：  2
聚类后，第一类list的长度为：  5
聚类后，第一类list的长度为：  7
聚类后，两层列表总长度为：  17
调用 k-means，聚类前list的长度为：  286
参与聚类的list 的长度 286
聚类后，第一类list的长度为：  286
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  286
调用 k-means，聚类前list的长度为：  11
参与聚类的list 的长度 11
聚类后，第一类list的长度为：  2
聚类后，第一类list的长

In [7]:
filename = ".\\os_file\\st_poi_clusters\\s_cluster\\st_poi_cluster_4nd.pkl"
st_poi = read_q(filename)
st_poi["enter"]["bar"]

[[{'id': '大堂吧',
   'location': {'lat': 39.996445, 'lng': 116.312735},
   'staying': [77.04999993788078],
   'category': 'bar'},
  {'id': '酒吧',
   'location': {'lat': 39.975998, 'lng': 116.32942800000001},
   'staying': [14.19999997713603,
    20.000000012805685,
    20.016666578594595,
    14.400000002933666,
    10.516666610492393,
    12.366666707675904,
    46.233333349227905,
    20.39999990724027,
    14.499999932013452,
    14.466666674707085,
    109.61666668416001,
    90.96666666213423,
    141.46666675340384,
    31.41666661365889,
    48.01666665938683,
    10.516666610492393,
    46.233333349227905,
    38.299999963492155,
    12.016666657291353,
    17.7333334449213,
    179.766666716896,
    26.900000054156408,
    27.949999974807724,
    23.416666702833027,
    16.35000003967434,
    18.53333324426785,
    10.516666610492393,
    13.29999998677522,
    144.3833333265502,
    23.916666673030704,
    31.049999861279503,
    215.68333334522322,
    224.7833332826849,
    23

In [12]:
filename = ".\\os_file\\st_poi_clusters\\mb_kmeans\\st_poi_cluster_1st.pkl"    
st_poi = read_q(filename)
st_poi["enter"]["bbulid"]

[[{'id': '北京大学附属小学-体育馆',
   'location': {'lat': 40.000085, 'lng': 116.32512},
   'staying': [212.31666662497446,
    180.0,
    21.683333372930065,
    99.79999992297962,
    172.9333332006354,
    27.00000000419095,
    196.2333332723938,
    82.84999997355044,
    50.7666666212026,
    45.36666676285677,
    41.433333264430985,
    182.74999997229315,
    333.04999996558763,
    143.0833334417548,
    212.31666662497446,
    180.0,
    21.683333372930065,
    99.79999992297962,
    172.9333332006354,
    27.00000000419095,
    196.2333332723938,
    82.84999997355044,
    50.7666666212026,
    45.36666676285677,
    132.18333335593343,
    306.6833332856186,
    126.46666670450941,
    144.58333334187046,
    13.250000016996637,
    222.03333331039175,
    221.56666660797782,
    67.59999992907979,
    94.09999996307306,
    78.2500000402797,
    156.9833333278075,
    215.98333328962326,
    340.3666666033678,
    106.3333333376795,
    179.4333332113456,
    122.26666665519588,
   

In [53]:
st_poi["enter"]["bath"]

[[{'id': '聚美足道',
   'location': {'lat': 40.010055, 'lng': 116.32128600000001},
   'staying': [15.416666645323858,
    13.583333229180425,
    13.983333270298317,
    13.983333270298317,
    12.650000107241794],
   'category': 'bath'},
  {'id': '圣心SPA会所(万柳店)',
   'location': {'lat': 39.973648, 'lng': 116.30124199999999},
   'staying': [17.21666663652286],
   'category': 'bath'},
  {'id': '华夏良子(三里屯店)',
   'location': {'lat': 39.933977, 'lng': 116.457462},
   'staying': [15.666666619945318, 26.299999861512333],
   'category': 'bath'},
  {'id': '丽府唐朝养生足道香薰SPA',
   'location': {'lat': 40.039467, 'lng': 116.284903},
   'staying': [21.616666701156646],
   'category': 'bath'},
  {'id': '殿凤专业修脚屋',
   'location': {'lat': 40.037079, 'lng': 116.344149},
   'staying': [14.333333331160247],
   'category': 'bath'},
  {'id': '翔天养生会所(角门店)',
   'location': {'lat': 39.851455, 'lng': 116.389905},
   'staying': [18.6500000115484, 19.266666769981384],
   'category': 'bath'},
  {'id': '大脚丫修脚(天秀路店)',
   'loca

33

In [56]:
filename = ".\\os_file\\st_poi_clusters\\mb_kmeans\\st_poi_cluster_2nd.pkl"
st_poi = read_q(filename)
st_poi["enter"]["bath"]

[[[{'id': '易柳元中医按摩',
    'location': {'lat': 39.969778999999996, 'lng': 116.30934199999999},
    'staying': [10.983333312906325,
     12.550000021001324,
     17.86666665226221,
     43.466666685417295,
     42.01666675508022,
     93.5500000230968,
     14.066666633589193,
     10.216666655614972,
     13.000000031897798,
     17.75000001071021,
     10.416666670935228],
    'category': 'bath'},
   {'id': '邢氏养生',
    'location': {'lat': 39.867129, 'lng': 116.486635},
    'staying': [12.583333435468376,
     15.799999973969534,
     19.4499999366235,
     12.416666687931865,
     20.400000053923577,
     34.66666669235565,
     12.583333435468376,
     35.3499999968335],
    'category': 'bath'},
   {'id': '晨光中医推拿',
    'location': {'lat': 40.077633, 'lng': 116.32786499999999},
    'staying': [21.71666664071381,
     47.81666664406657,
     11.233333298005164,
     26.53333329129964],
    'category': 'bath'}],
  [{'id': '聚美足道',
    'location': {'lat': 40.010055, 'lng': 116.3212860000000

In [15]:
filename = ".\\os_file\\st_poi_clusters\\mb_kmeans\\st_poi_cluster_3nd.pkl"
st_poi = read_q(filename)
st_poi["enter"]["bbulid"]

[[[[{'id': '北京大学附属小学-体育馆',
     'location': {'lat': 40.000085, 'lng': 116.32512},
     'staying': [212.31666662497446,
      180.0,
      21.683333372930065,
      99.79999992297962,
      172.9333332006354,
      27.00000000419095,
      196.2333332723938,
      82.84999997355044,
      50.7666666212026,
      45.36666676285677,
      41.433333264430985,
      182.74999997229315,
      333.04999996558763,
      143.0833334417548,
      212.31666662497446,
      180.0,
      21.683333372930065,
      99.79999992297962,
      172.9333332006354,
      27.00000000419095,
      196.2333332723938,
      82.84999997355044,
      50.7666666212026,
      45.36666676285677,
      132.18333335593343,
      306.6833332856186,
      126.46666670450941,
      144.58333334187046,
      13.250000016996637,
      222.03333331039175,
      221.56666660797782,
      67.59999992907979,
      94.09999996307306,
      78.2500000402797,
      156.9833333278075,
      215.98333328962326,
      340.3666666033

In [17]:
filename = ".\\os_file\\st_poi_clusters\\mb_kmeans\\st_poi_cluster_4nd.pkl"
st_poi = read_q(filename)
st_poi["enter"]["bar"]

[[{'id': '大堂吧',
   'location': {'lat': 39.996445, 'lng': 116.312735},
   'staying': [77.04999993788078],
   'category': 'bar'}],
 [[[[{'id': '京都信苑饭店-大堂酒吧',
      'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
      'staying': [12.249999940395355],
      'category': 'bar'},
     {'id': '顽啤猴精酿(西山摩尔汇店)',
      'location': {'lat': 39.934197, 'lng': 116.103996},
      'staying': [19.033333350671455],
      'category': 'bar'},
     {'id': '北京中关村皇冠假日酒店-大堂酒廊',
      'location': {'lat': 39.981415999999996, 'lng': 116.329609},
      'staying': [23.7500000721775, 10.083333301590756],
      'category': 'bar'},
     {'id': '九霄云外酒吧(香格里拉饭店)',
      'location': {'lat': 39.950293, 'lng': 116.31490600000001},
      'staying': [14.500000078696758],
      'category': 'bar'},
     {'id': 'seven bar',
      'location': {'lat': 39.948612, 'lng': 116.40956399999999},
      'staying': [11.916666717734188],
      'category': 'bar'},
     {'id': '星巴克(腾达大厦店)',
      'location': {'lat': 39.943

In [18]:
st_poi["enter"]["bbulid"]

[[[[[{'id': '美树瑜伽(北京友谊宾馆店)',
      'location': {'lat': 39.971121000000004, 'lng': 116.326844},
      'staying': [60.76666670618579,
       14.750000063795596,
       61.18333329213783,
       18.46666670870036],
      'category': 'bbulid'},
     {'id': '篮球场',
      'location': {'lat': 40.057014, 'lng': 116.332926},
      'staying': [24.166666668606922, 67.58333337376826],
      'category': 'bbulid'},
     {'id': '海淀公园体育中心',
      'location': {'lat': 39.990522999999996, 'lng': 116.298948},
      'staying': [11.999999955296516,
       12.416666687931865,
       61.58333333325572,
       11.299999969778582,
       66.43333338783123,
       71.13333327113651],
      'category': 'bbulid'},
     {'id': '地中海俱乐部长城度假村体验中心',
      'location': {'lat': 39.989398, 'lng': 116.45064599999999},
      'staying': [65.2833334018942,
       38.31666667596437,
       12.41666654124856,
       65.00000001280569,
       12.066666627069935],
      'category': 'bbulid'},
     {'id': '奇迹空间健身会所(外馆斜街)',
      'lo

In [34]:
a = st_poi["enter"]["bar"][0][0][0]
a

[{'id': '京都信苑饭店-大堂酒吧',
  'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
  'staying': [12.249999940395355],
  'category': 'bar'},
 {'id': '顽啤猴精酿(西山摩尔汇店)',
  'location': {'lat': 39.934197, 'lng': 116.103996},
  'staying': [19.033333350671455],
  'category': 'bar'},
 {'id': '北京中关村皇冠假日酒店-大堂酒廊',
  'location': {'lat': 39.981415999999996, 'lng': 116.329609},
  'staying': [23.7500000721775, 10.083333301590756],
  'category': 'bar'},
 {'id': '九霄云外酒吧(香格里拉饭店)',
  'location': {'lat': 39.950293, 'lng': 116.31490600000001},
  'staying': [14.500000078696758],
  'category': 'bar'},
 {'id': 'seven bar',
  'location': {'lat': 39.948612, 'lng': 116.40956399999999},
  'staying': [11.916666717734188],
  'category': 'bar'},
 {'id': '星巴克(腾达大厦店)',
  'location': {'lat': 39.943732000000004, 'lng': 116.333277},
  'staying': [14.949999932432547],
  'category': 'bar'},
 {'id': '万欧兰葡萄酒俱乐部',
  'location': {'lat': 39.905714, 'lng': 116.475268},
  'staying': [11.466666717315093],
  'category': 'ba

In [43]:
b = [a,[],[],[]]
b

[[{'id': '京都信苑饭店-大堂酒吧',
   'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
   'staying': [12.249999940395355],
   'category': 'bar'},
  {'id': '顽啤猴精酿(西山摩尔汇店)',
   'location': {'lat': 39.934197, 'lng': 116.103996},
   'staying': [19.033333350671455],
   'category': 'bar'},
  {'id': '北京中关村皇冠假日酒店-大堂酒廊',
   'location': {'lat': 39.981415999999996, 'lng': 116.329609},
   'staying': [23.7500000721775, 10.083333301590756],
   'category': 'bar'},
  {'id': '九霄云外酒吧(香格里拉饭店)',
   'location': {'lat': 39.950293, 'lng': 116.31490600000001},
   'staying': [14.500000078696758],
   'category': 'bar'},
  {'id': 'seven bar',
   'location': {'lat': 39.948612, 'lng': 116.40956399999999},
   'staying': [11.916666717734188],
   'category': 'bar'},
  {'id': '星巴克(腾达大厦店)',
   'location': {'lat': 39.943732000000004, 'lng': 116.333277},
   'staying': [14.949999932432547],
   'category': 'bar'},
  {'id': '万欧兰葡萄酒俱乐部',
   'location': {'lat': 39.905714, 'lng': 116.475268},
   'staying': [11.46666671

In [47]:
a = mb_kmeans(b[0],4)
a

调用 mb_kmeans，聚类前list的长度为：  16
参与聚类的list 的长度 16
聚类后，第一类list的长度为：  16
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，第一类list的长度为：  0
聚类后，两层列表总长度为：  16


[[{'id': '京都信苑饭店-大堂酒吧',
   'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
   'staying': [12.249999940395355],
   'category': 'bar'},
  {'id': '顽啤猴精酿(西山摩尔汇店)',
   'location': {'lat': 39.934197, 'lng': 116.103996},
   'staying': [19.033333350671455],
   'category': 'bar'},
  {'id': '北京中关村皇冠假日酒店-大堂酒廊',
   'location': {'lat': 39.981415999999996, 'lng': 116.329609},
   'staying': [23.7500000721775, 10.083333301590756],
   'category': 'bar'},
  {'id': '九霄云外酒吧(香格里拉饭店)',
   'location': {'lat': 39.950293, 'lng': 116.31490600000001},
   'staying': [14.500000078696758],
   'category': 'bar'},
  {'id': 'seven bar',
   'location': {'lat': 39.948612, 'lng': 116.40956399999999},
   'staying': [11.916666717734188],
   'category': 'bar'},
  {'id': '星巴克(腾达大厦店)',
   'location': {'lat': 39.943732000000004, 'lng': 116.333277},
   'staying': [14.949999932432547],
   'category': 'bar'},
  {'id': '万欧兰葡萄酒俱乐部',
   'location': {'lat': 39.905714, 'lng': 116.475268},
   'staying': [11.46666671

In [41]:
del(a[16])
a

[{'id': '京都信苑饭店-大堂酒吧',
  'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
  'staying': [12.249999940395355],
  'category': 'bar'},
 {'id': '顽啤猴精酿(西山摩尔汇店)',
  'location': {'lat': 39.934197, 'lng': 116.103996},
  'staying': [19.033333350671455],
  'category': 'bar'},
 {'id': '北京中关村皇冠假日酒店-大堂酒廊',
  'location': {'lat': 39.981415999999996, 'lng': 116.329609},
  'staying': [23.7500000721775, 10.083333301590756],
  'category': 'bar'},
 {'id': '九霄云外酒吧(香格里拉饭店)',
  'location': {'lat': 39.950293, 'lng': 116.31490600000001},
  'staying': [14.500000078696758],
  'category': 'bar'},
 {'id': 'seven bar',
  'location': {'lat': 39.948612, 'lng': 116.40956399999999},
  'staying': [11.916666717734188],
  'category': 'bar'},
 {'id': '星巴克(腾达大厦店)',
  'location': {'lat': 39.943732000000004, 'lng': 116.333277},
  'staying': [14.949999932432547],
  'category': 'bar'},
 {'id': '万欧兰葡萄酒俱乐部',
  'location': {'lat': 39.905714, 'lng': 116.475268},
  'staying': [11.466666717315093],
  'category': 'ba